<a href="https://colab.research.google.com/github/SridharSola/Knowledge-Distillation-FER/blob/main/GradCAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This method obtains the heatmap for gradients on the train set (or set given), superimposes the heatmap
on the original image, and saves the superimposed image as a jpg file in a folder on the drive
"""
def GradCAM(net, train_loader):
  net.cuda()
  net.eval()

  for j, (imgu, imgm, l, p1, p2) in enumerate(train_loader):
      imgu = to_device(imgu, device)
      pred1 = net(imgu)
      indx = pred1.argmax(dim = 1)
      # get the gradient of the output with respect to the parameters of the model
      pred1[:, indx].backward()

      # pull the gradients out of the model
      gradients = net.get_activations_gradient()

      # pool the gradients across the channels
      pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

      # get the activations of the last convolutional layer
      activations = net.get_activations(imgu).detach()

      # weight the channels by corresponding gradients
      for i in range(512):
          activations[:, i, :, :] *= pooled_gradients[i]
          
      # average the channels of the activations
      heatmap = torch.mean(activations, dim=1).squeeze().cpu()
      # relu on top of the heatmap
      # expression (2) in https://arxiv.org/pdf/1610.02391.pdf
      heatmap = np.maximum(heatmap, 0)
      #Superimposing
      img = cv2.imread(p1[0])
      # normalize the heatmap
      heatmap /= torch.max(heatmap)
      heatmap = np.uint8(255 * heatmap)
      heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
      heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
      superimposed_img = heatmap * 0.4 + img
      #cv2.imwrite('./map.jpg', superimposed_img)
      if j%100:
          cv2_imshow(superimposed_img)
      # draw the heatmap
      #plt.matshow(heatmap.squeeze())
      cv2.imwrite('/content/drive/MyDrive/FER_Project/GradCam/' + str(j) + '.jpg', superimposed_img)